In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# read in the file downloaded from google drive where we manually labelled
sample = pd.read_csv('../data/training/new_sample_111219.csv', index_col= 0)
# get rid of commas
sample['ADDR_NUM'] = sample['ADDR_NUM'].replace(',', '', regex=True)
# one off fix for this crazy address
sample.loc[sample.ADDR_NUM == '43717', 'ADDR_NUM'] = '9-9'

In [3]:
## removing this one bc it exists in gsv not in aerial
sample.drop(sample[sample.ADDR_NUM == '429'].index, inplace = True)

In [4]:
sample.head()

,ADDR_NUM,FULL_STR,MBL,ADDRESS_FOR_STREET_VIEW,AERIAL_Driveway,GSV_Driveway
8041,5,DOUGLAS AVE,47-G-17,"5 DOUGLAS AVE, SOMERVILLE, MA",2.0,0.0
6488,6,HILLSIDE CIR,38-A-47,"6 HILLSIDE CIR, SOMERVILLE, MA",1.0,2.0
7629,8,BELMONT PL,44-A-48,"8 BELMONT PL, SOMERVILLE, MA",2.0,2.0
7669,8,DRESDEN CIR,25-A-13,"8 DRESDEN CIR, SOMERVILLE, MA",2.0,2.0
7706,8,HAWTHORNE ST,25-D-16,"8 HAWTHORNE ST, SOMERVILLE, MA",0.0,2.0


In [5]:
sample = sample[~sample.AERIAL_Driveway.isna()]
sample = sample[~sample.GSV_Driveway.isna()]

### labels
- 0 = no driveway 
- 1 = driveway 
- 2 = unsure

In [6]:
def overall_labelling(y):
    y1, y2= y[0], y[1]
    
    ### ALL CERTAIN ###
    # both have driveways
    if y1 ==1.0 and y2 == 1.0:
        return 1.0
    # both don't have driveways
    if y1 == 0.0 and y2 == 0.0:
        return 0.0
    # aerial has driveway, streetview doesn't have driveway
    if y1 ==1.0 and y2 == 0.0:
        return 0.9
    # aerial doesn't have driveway, streetview has driveway
    if y1 == 0.0 and y2 == 1.0:
        return 0.9
    
    ### ALL UNCERTAIN ###
    if y1 == 2.0 and y2 == 2.0:
        return 0.5
    
    ### ONE UNCERTAIN ###

    if y1 == 2.0 and y2 != 2.0:
        if y2 == 1.0:
            return 0.9
        if y2 == 0.0:
            return 0.1
    
    if y1 != 2.0 and y2 == 2.0:
        if y1 == 1.0:
            return 0.9
        if y1 == 0.0:
            return 0.1

In [7]:
sample['final_label'] = sample[['AERIAL_Driveway','GSV_Driveway']].apply(overall_labelling, axis = 1)

In [8]:
sample['aerial_filename'] = sample['ADDR_NUM'] + '_' + sample['FULL_STR'].str.replace(' ', '_') + '_aerial.png'
sample['gsv_filename'] = sample['ADDR_NUM'] + '_' + sample['FULL_STR'].str.replace(' ', '_') + '.jpg'

In [9]:
sample.head()

,ADDR_NUM,FULL_STR,MBL,ADDRESS_FOR_STREET_VIEW,AERIAL_Driveway,GSV_Driveway,final_label,aerial_filename,gsv_filename
8041,5,DOUGLAS AVE,47-G-17,"5 DOUGLAS AVE, SOMERVILLE, MA",2.0,0.0,0.1,5_DOUGLAS_AVE_aerial.png,5_DOUGLAS_AVE.jpg
6488,6,HILLSIDE CIR,38-A-47,"6 HILLSIDE CIR, SOMERVILLE, MA",1.0,2.0,0.9,6_HILLSIDE_CIR_aerial.png,6_HILLSIDE_CIR.jpg
7629,8,BELMONT PL,44-A-48,"8 BELMONT PL, SOMERVILLE, MA",2.0,2.0,0.5,8_BELMONT_PL_aerial.png,8_BELMONT_PL.jpg
7669,8,DRESDEN CIR,25-A-13,"8 DRESDEN CIR, SOMERVILLE, MA",2.0,2.0,0.5,8_DRESDEN_CIR_aerial.png,8_DRESDEN_CIR.jpg
7706,8,HAWTHORNE ST,25-D-16,"8 HAWTHORNE ST, SOMERVILLE, MA",0.0,2.0,0.1,8_HAWTHORNE_ST_aerial.png,8_HAWTHORNE_ST.jpg


In [10]:
sample.to_csv('../labels/training_labels_updated_111219.csv', index=False)

---